In [1]:
%autoreload 2

[autoreload of src.fitness_energy_utils failed: Traceback (most recent call last):
  File "/Users/guydavidson/opt/anaconda3/envs/game-gen/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 261, in check
    superreload(m, reload, self.old_objects)
  File "/Users/guydavidson/opt/anaconda3/envs/game-gen/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 484, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/guydavidson/opt/anaconda3/envs/game-gen/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 381, in update_generic
    update(a, b)
  File "/Users/guydavidson/opt/anaconda3/envs/game-gen/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 293, in update_function
    setattr(old, name, getattr(new, name))
ValueError: evaluate_fitness_overall_ecdf() requires a code object with 0 free vars, not 1
]


In [2]:
from collections import defaultdict
from argparse import Namespace
import copy
import gzip
import itertools
import os
import pickle
import sys
import typing

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tatsu
import tatsu.ast
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import tqdm
import sklearn
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.pipeline import Pipeline

sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))
from src.ast_utils import _extract_game_id
from src import fitness_energy_utils as utils
from src.fitness_energy_utils import NON_FEATURE_COLUMNS
from src.fitness_features import *
from src.ast_counter_sampler import *
from src.ast_mcmc_regrowth import *

In [3]:
grammar = open('../dsl/dsl.ebnf').read()
grammar_parser = tatsu.compile(grammar)
game_asts = list(cached_load_and_parse_games_from_file('../dsl/interactive-beta.pddl', grammar_parser, False, relative_path='..'))
real_game_texts = [ast_printer.ast_to_string(ast, '\n') for ast in game_asts]
regrown_game_texts = list(load_games_from_file('../dsl/ast-real-regrowth-samples.pddl'))
regrown_game_1024_texts = list(load_games_from_file('../dsl/ast-real-regrowth-samples-1024.pddl'))
print(len(real_game_texts), len(regrown_game_texts), len(regrown_game_texts) / 98, len(regrown_game_1024_texts), len(regrown_game_1024_texts) / 98)


98 12544 128.0 100352 1024.0


In [4]:
fitness_df = utils.load_fitness_data('../data/fitness_features_1024_regrowths.csv.gz')
print(fitness_df.src_file.unique())
fitness_df.head()

['interactive-beta.pddl' 'ast-real-regrowth-samples-1024.pddl']


Index               src_file                   game_name  ast_ngram_n_7_8  \
0      0  interactive-beta.pddl  6172feb1665491d1efbce164-0                0   
1      1  interactive-beta.pddl  5f77754ba932fb2c4ba181d8-2                1   
2      2  interactive-beta.pddl  614b603d4da88384282967a7-3                0   
3      3  interactive-beta.pddl  5bc79f652885710001a0e82a-5                1   
4      4  interactive-beta.pddl  614dec67f6eb129c3a77defd-6                1   

   ast_ngram_n_7_6  ast_ngram_n_7_7  ast_ngram_n_7_9             domain_name  \
0                0                0                0  medium-objects-room-v1   
1                1                1                0    many-objects-room-v1   
2                0                0                0    many-objects-room-v1   
3                1                1                0     few-objects-room-v1   
4                1                1                0  medium-objects-room-v1   

   real          original_game_name  ...  ast_ngram_n_6_6  ast_ngram_n_6_7  \
0     1  6172feb1665491d1efbce164-0  ...                1                0   
1     1  5f77754ba932fb2c4ba181d8-2  ...                0                0   
2     1  614b603d4da88384282967a7-3  ...                0                0   
3     1  5bc79f652885710001a0e82a-5  ...                1                0   
4     1  614dec67f6eb129c3a77defd-6  ...                1                0   

   ast_ngram_n_6_8  ast_ngram_n_6_9  ast_ngram_n_7_0  ast_ngram_n_7_1  \
0                0                0                0                0   
1                1                1                1                1   
2                0                0                0                0   
3                1                1                1                0   
4                1                1                1                1   

   ast_ngram_n_7_2  ast_ngram_n_7_3  ast_ngram_n_7_4  ast_ngram_n_7_5  
0                1                1                0                0  
1                0                0                1                1  
2                0                0                0                0  
3                1                1                1                1  
4                1                1                1                1  

[5 rows x 578 columns]

# Plan of attack
* Use the code in `ast_counter_sampler.py` to generate a sample from the MLE
* Score it with a fitness function adapted to working with a single example at a time
* At each subsequent iteration, regrow the game from a random node, and score the regrowth
* Since we have energy scores before and after, we can either accept greedily if $E_{new} < E_{old}$, or accept with probability $\exp(\beta (E_{old} - E_{new} ))$

In [5]:
DEFAULT_ARGS = argparse.Namespace(
    grammar_file=os.path.join('..', DEFAULT_GRAMMAR_FILE),
    parse_counter=False,
    counter_output_path=os.path.join('..', DEFAULT_COUNTER_OUTPUT_PATH),
    random_seed=DEFUALT_RANDOM_SEED,
)

In [6]:
FITNESS_MODEL_PATH = '../models/cv_fitness_model_2023_02_21.pkl.gz'
FITNESS_FEATURIZER_PATH = '../models/fitness_featurizer_2023_02_20.pkl.gz'

feature_columns = [str(c) for c in fitness_df.columns if c not in NON_FEATURE_COLUMNS]
for n in range(2, 7):
    feature_columns.remove(f'ast_ngram_n_{n}_score')

mcmc = MCMCRegrowthSampler(DEFAULT_ARGS, feature_columns,
    FITNESS_MODEL_PATH, FITNESS_FEATURIZER_PATH, greedy_acceptance=True, 
    plateau_patience_steps=100, max_steps=10000)  #   acceptance_temperature=10.0, 

In [11]:
n_samples = 10
mcmc.multiple_samples(n_samples, verbose=1, should_tqdm=True)

  0%|          | 0/10 [00:00<?, ?it/s]

Plateaued at step 704 with energy 2.65309
Plateaued at step 1240 with energy 8.17084
Plateaued at step 2039 with energy 4.49677
Plateaued at step 641 with energy 0.80883
Plateaued at step 780 with energy -2.07716
Plateaued at step 527 with energy 1.74399
Plateaued at step 611 with energy 4.48507
Plateaued at step 498 with energy 5.18139
Plateaued at step 537 with energy 1.61536
Plateaued at step 263 with energy 7.30220


In [43]:
mcmc.visualize_sample(5)

### Energy of visualized game: 1.744

### Top features contributing to the game's energy
feature name: **value** = (original feature value => regrown feature value) * weight

| Features increasing energy (= more fake)                                       | Features decreasing energy (= more real)                                          |
|--------------------------------------------------------------------------------|-----------------------------------------------------------------------------------|
| ast_ngram_n_2_4: **2.689** = (1.000) * 2.689                                   | all_variables_used: **-3.505** = (1.000) * -3.505                                 |
| no_identical_logical_children: **1.797** = (1.000) * 1.797                     | variable_not_repeated: **-2.998** = (1.000) * -2.998                              |
| node_count_terminal_2: **1.187** = (1.000) * 1.187                             | ast_ngram_n_7_score: **-2.283** = (0.477) * -4.790                                |
| node_count_setup_3: **0.990** = (1.000) * 0.990                                | all_preferences_used: **-1.728** = (1.000) * -1.728                               |
| max_quantification_count_constraints_1: **0.940** = (1.000) * 0.940            | no_two_number_operations: **-1.020** = (1.000) * -1.020                           |
| max_depth_setup_2: **0.853** = (1.000) * 0.853                                 | no_nested_logicals: **-0.931** = (1.000) * -0.931                                 |
| mean_depth_constraints_1: **0.776** = (1.000) * 0.776                          | ast_ngram_n_2_5: **-0.712** = (1.000) * -0.712                                    |
| mean_depth_setup_2: **0.769** = (1.000) * 0.769                                | no_adjacent_same_modal: **-0.675** = (1.000) * -0.675                             |
| max_number_variables_types_quantified_variables_1: **0.754** = (1.000) * 0.754 | distance_arg_types_balls_room_features_setup: **-0.420** = (1.000) * -0.420       |
| node_count_constraints_0: **0.729** = (1.000) * 0.729                          | all_variables_defined: **-0.392** = (1.000) * -0.392                              |
| ast_ngram_n_2_6: **0.527** = (1.000) * 0.527                                   | in_motion_arg_types_large_objects_constraints: **-0.357** = (1.000) * -0.357      |
| max_depth_constraints_1: **0.513** = (1.000) * 0.513                           | distance_arg_types_room_features_receptacles_setup: **-0.334** = (1.000) * -0.334 |
| ast_ngram_n_4_9: **0.377** = (1.000) * 0.377                                   | node_count_scoring_0: **-0.241** = (1.000) * -0.241                               |
| mean_depth_terminal_2: **0.348** = (1.000) * 0.348                             |                                                                                   |
| max_quantification_count_setup_2: **0.279** = (1.000) * 0.279                  |                                                                                   |
| section_without_pref_or_total_count_scoring: **0.255** = (1.000) * 0.255       |                                                                                   |
| agent_holds_arg_types_large_objects_constraints: **0.201** = (1.000) * 0.201   |                                                                                   |
| max_depth_scoring_0: **0.164** = (1.000) * 0.164                               |                                                                                   |
| ast_ngram_n_2_8: **0.164** = (1.000) * 0.164                                   |                                                                                   |
| length_of_then_modals_2: **0.112** = (1.000) * 0.112                           |                                                                                   |

### Game:

```pddl
(define (game mcmc-5) (:domain few-objects-room-v1)
(:setup
  (exists (?v0 - red_dodgeball)
    (and
      (exists (?v1 - (either dodgeball chair curved_wooden_ramp wall book alarm_clock dodgeball) ?v1 - hexagonal_bin)
        (game-optional
          (< (distance ?v0 west_wall) (distance floor ?v1))
        )
      )
    )
  )
)
(:constraints
  (and
    (preference preference0
      (exists (?v2 - pillow)
        (then
          (once (not (agent_holds ?v2) ) )
          (hold (and (agent_holds ?v2) (in_motion ?v2) ) )
        )
      )
    )
  )
)
(:terminal
  (>= (count preference0) 1 )
)
(:scoring
  6
)
)
```

In [44]:
{k:v for k, v in mcmc.samples[5][1].items() if 'in_motion' in k}

{'in_motion_arg_types_agent_setup': 0,
 'in_motion_arg_types_agent_constraints': 0,
 'in_motion_arg_types_any_object_setup': 0,
 'in_motion_arg_types_any_object_constraints': 0,
 'in_motion_arg_types_balls_setup': 0,
 'in_motion_arg_types_balls_constraints': 0,
 'in_motion_arg_types_blocks_setup': 0,
 'in_motion_arg_types_blocks_constraints': 0,
 'in_motion_arg_types_building_constraints': 0,
 'in_motion_arg_types_colors_constraints': 0,
 'in_motion_arg_types_furniture_setup': 0,
 'in_motion_arg_types_furniture_constraints': 0,
 'in_motion_arg_types_large_objects_setup': 0,
 'in_motion_arg_types_large_objects_constraints': 1,
 'in_motion_arg_types_orientations_constraints': 0,
 'in_motion_arg_types_other_setup': 0,
 'in_motion_arg_types_ramps_setup': 0,
 'in_motion_arg_types_ramps_constraints': 0,
 'in_motion_arg_types_receptacles_setup': 0,
 'in_motion_arg_types_receptacles_constraints': 0,
 'in_motion_arg_types_room_features_setup': 0,
 'in_motion_arg_types_room_features_constraints'

In [30]:
s = '''(define (game mcmc-5) (:domain few-objects-room-v1)\n(:setup\n  (exists (?v0 - red_dodgeball)\n    (and\n      (exists (?v1 - (either dodgeball chair curved_wooden_ramp wall book alarm_clock dodgeball) ?v1 - hexagonal_bin)\n        (game-optional\n          (< (distance ?v0 west_wall) (distance floor ?v1))\n        )\n      )\n    )\n  )\n)\n(:constraints\n  (and\n    (preference preference0\n      (exists (?v2 - pillow)\n        (then\n          (once (not (agent_holds ?v2) ) )\n          (hold (and (agent_holds ?v2) (in_motion ?v2) ) )\n        )\n      )\n    )\n  )\n)\n(:terminal\n  (>= (count preference0) 1 )\n)\n(:scoring\n  6\n)\n)'''
print(s)

(define (game mcmc-5) (:domain few-objects-room-v1)
(:setup
  (exists (?v0 - red_dodgeball)
    (and
      (exists (?v1 - (either dodgeball chair curved_wooden_ramp wall book alarm_clock dodgeball) ?v1 - hexagonal_bin)
        (game-optional
          (< (distance ?v0 west_wall) (distance floor ?v1))
        )
      )
    )
  )
)
(:constraints
  (and
    (preference preference0
      (exists (?v2 - pillow)
        (then
          (once (not (agent_holds ?v2) ) )
          (hold (and (agent_holds ?v2) (in_motion ?v2) ) )
        )
      )
    )
  )
)
(:terminal
  (>= (count preference0) 1 )
)
(:scoring
  6
)
)


In [25]:
from argparse import Namespace
from src.fitness_features import *
args = Namespace(no_binarize=False, no_merge=False)
featurizer = build_fitness_featurizer(args)

In [26]:
a = grammar_parser.parse(s)

In [28]:
ps = {k:v for k, v in featurizer.parse(a, 'mcmc', True).items() if 'predicate_under_modal' in k}
len(ps)

50

In [32]:
{k:v for k, v in ps.items() if v > 0}

{'predicate_under_modal_hold_in_motion': 1,
 'predicate_under_modal_hold_agent_holds': 1,
 'predicate_under_modal_once_agent_holds': 1}

In [51]:
mcmc.samples[5][0][3][1].setup.exists_args.setup.and_args[0].setup.exists_vars

{'variables': [{'var_names': ['?v1'], 'var_type': {'type': {'type_names': ['dodgeball', 'chair', 'curved_wooden_ramp', 'wall', 'book', 'alarm_clock', 'dodgeball'], 'parseinfo': {'tokenizer': None, 'rule': 'either_types', 'pos': 3357, 'endpos': 3357, 'line': 3357, 'endline': 3357, 'alerts': []}}, 'parseinfo': {'tokenizer': None, 'rule': 'type_definition', 'pos': 3358, 'endpos': 3358, 'line': 3358, 'endline': 3358, 'alerts': []}}, 'parseinfo': {'tokenizer': None, 'rule': 'variable_type_def', 'pos': 3359, 'endpos': 3359, 'line': 3359, 'endline': 3359, 'alerts': []}}, {'var_names': ['?v1'], 'var_type': {'type': 'hexagonal_bin', 'parseinfo': {'tokenizer': None, 'rule': 'type_definition', 'pos': 7, 'endpos': 7, 'line': 7, 'endline': 7, 'alerts': []}}, 'parseinfo': {'tokenizer': None, 'rule': 'variable_type_def', 'pos': 8, 'endpos': 8, 'line': 8, 'endline': 8, 'alerts': []}}], 'parseinfo': {'tokenizer': None, 'rule': 'variable_list', 'pos': 9, 'endpos': 9, 'line': 9, 'endline': 9, 'alerts': [

In [ ]:
mcmc._proposal_to_features(mcmc.samples[0][0])['ast_ngram_n_7_score']

In [ ]:
mcmc.fitness_featurizer.full_ast_registry[0].n_gram_model.score(mcmc.samples[0][0], log=True)

In [ ]:
mcmc.fitness_featurizer.preprocessors[0].scale_series_min_max_values

In [ ]:
# %prun -r 
n_samples = 5
start = 1
end = start + 5

for original_index in range(start, end):
    print(f'Generating samples starting from original index {original_index} (id {_extract_game_id(real_game_texts[original_index])})')
    mcmc.multiple_samples(n_samples, verbose=1, should_tqdm=True, initial_proposal=game_asts[original_index])

    print()
    for i in range(n_samples * (original_index - start), n_samples * (original_index - start + 1)):
        print(i, mcmc.samples[i][2])

In [ ]:
mcmc.visualize_sample(14)

In [ ]:
mcmc.fitness_featurizer.full_text_registry[0].n_gram_model.models[-1].score(ast_printer.ast_to_string(mcmc.samples[14][0], ' '))

In [ ]:
mcmc.fitness_featurizer.full_text_registry[0].n_gram_model.models[-1].score(ast_printer.ast_to_string(mcmc.samples[14][3], ' '))

In [ ]:
ngram_model = mcmc.fitness_featurizer.full_text_registry[0].n_gram_model.models[-1]

sample_ngrams = [('constraints', 'and', 'forall', 'wall', 'and'), ('and', 'forall', 'wall', 'and', 'preference'), ('forall', 'wall', 'and', 'preference', 'preferenceName'), ('wall', 'and', 'preference', 'preferenceName', 'exists'), ('and', 'preference', 'preferenceName', 'exists', 'block'), ('preference', 'preferenceName', 'exists', 'block', 'atend'), ('preferenceName', 'exists', 'block', 'atend', 'in'), ('exists', 'block', 'atend', 'in', 'scoring'), ('block', 'atend', 'in', 'scoring', 'externalforallmaximize'), ('atend', 'in', 'scoring', 'externalforallmaximize', 'countonceperobjects'), ('in', 'scoring', 'externalforallmaximize', 'countonceperobjects', 'preferenceName')]
original_ngrams = [('constraints', 'and', 'forall', 'building', 'and'), ('and', 'forall', 'building', 'and', 'preference'), ('forall', 'building', 'and', 'preference', 'preferenceName'), ('building', 'and', 'preference', 'preferenceName', 'exists'), ('and', 'preference', 'preferenceName', 'exists', 'block'), ('preference', 'preferenceName', 'exists', 'block', 'atend'), ('preferenceName', 'exists', 'block', 'atend', 'in'), ('exists', 'block', 'atend', 'in', 'scoring'), ('block', 'atend', 'in', 'scoring', 'externalforallmaximize'), ('atend', 'in', 'scoring', 'externalforallmaximize', 'countonceperobjects'), ('in', 'scoring', 'externalforallmaximize', 'countonceperobjects', 'preferenceName')]

print(f'Sample ngrams: {np.mean([ngram_model.ngram_logprobs[s] for s in sample_ngrams])} | ')
for s in sample_ngrams:
    print(s, ngram_model.ngram_counts[s], ngram_model.ngram_logprobs[s])

print()
print(f'Original ngrams: {np.mean([ngram_model.ngram_logprobs[s] for s in original_ngrams])}')
for s in original_ngrams:
    print(s, ngram_model.ngram_counts[s], ngram_model.ngram_logprobs[s])
print()

for s in sample_ngrams[:]:
    if s in original_ngrams:
        sample_ngrams.remove(s)
        original_ngrams.remove(s)

print('Sample remaining ngrams')
for s in sample_ngrams:
    print(s, ngram_model.ngram_counts[s], ngram_model.ngram_logprobs[s])

print()
print('Original remaining ngrams')
for s in original_ngrams:
    print(s, ngram_model.ngram_counts[s], ngram_model.ngram_logprobs[s])

In [ ]:
original_ngrams = [('constraints', 'and', 'forall', 'building', 'and'), ('and', 'forall', 'building', 'and', 'preference'), ('forall', 'building', 'and', 'preference', 'preferenceName'), ('building', 'and', 'preference', 'preferenceName', 'exists'), ('and', 'preference', 'preferenceName', 'exists', 'block'), ('preference', 'preferenceName', 'exists', 'block', 'atend'), ('preferenceName', 'exists', 'block', 'atend', 'in'), ('exists', 'block', 'atend', 'in', 'scoring'), ('block', 'atend', 'in', 'scoring', 'externalforallmaximize'), ('atend', 'in', 'scoring', 'externalforallmaximize', 'countonceperobjects'), ('in', 'scoring', 'externalforallmaximize', 'countonceperobjects', 'preferenceName')]
print('Original ngrams')
for s in original_ngrams:
    print(s, mcmc.fitness_featurizer.full_text_registry[0].n_gram_model.models[-1].ngram_counts[s], mcmc.fitness_featurizer.full_text_registry[0].n_gram_model.models[-1].ngram_logprobs[s])

In [ ]:
mcmc.visualize_sample(7)

In [ ]:
mcmc.visualize_sample(15)

### Notes from looking at these games
* **TODO:** debug the 'section-without' logic, it seems to not work
* **TODO:** add features for preferences with different lengths of modals

In [ ]:
print(ast_printer.ast_to_string(samples[8][0], '\n'))

In [ ]:
fitness_featurizer = build_fitness_featurizer(DEFAULT_ARGS)
features = fitness_featurizer.parse(samples[7][0], 'mcmc', True)
{k: v for k, v in features.items() if 'correctly' in k or 'forall' in k or 'no' in k}

In [ ]:
with gzip.open(DEFAULT_FITNESS_FUNCTION_PATH, 'rb') as f:
    cv_fitness_model = pickle.load(f)

In [ ]:
weights = cv_fitness_model.named_steps['fitness'].model.fc1.weight.data.detach().numpy().squeeze()
bias = cv_fitness_model.named_steps['fitness'].model.fc1.bias.data.detach().numpy().squeeze()
print(weights.mean(), bias)

In [ ]:
plt.hist(weights, bins=100)
plt.title('Fitness Model Weights')
plt.xlabel('Weight magnitude')
plt.ylabel('Count')

In [ ]:
import ast_crossover_sampler
asts = [ast for ast in cached_load_and_parse_games_from_file('../dsl/interactive-beta.pddl', 
    mcmc.grammar_parser, False, relative_path='..')]

mcmc_with_crossover = MCMCRegrowthCrossoverSampler(DEFAULT_ARGS, 
    ast_crossover_sampler.CrossoverType.SAME_RULE, asts, 0.25,
    '../models/cv_fitness_model_2022_12_26.pkl.gz', 
    greedy_acceptance=True, plateau_patience_steps=20, acceptance_temperature=10.0, max_steps=1000)

In [ ]:
mcmc_with_crossover.multiple_samples(10, verbose=2, should_tqdm=True)
print([x[2] for x in mcmc_with_crossover.samples])

In [ ]:
print([x[2] for x in mcmc_with_crossover.samples])